## 1. Importar Librerías Necesarias

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

# Para mostrar todas las columnas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

## 2. Cargar y Explorar el Dataset

In [ ]:
# Cargar el dataset del Titanic
df = pd.read_csv('Titanic-Dataset.csv')

# Mostrar primeras filas
print("Primeras 5 filas del dataset:")
df.head()

In [ ]:
# Información general del dataset
print("Información del dataset:")
df.info()

In [ ]:
# Dimensiones del dataset
print(f"Dimensiones del dataset: {df.shape}")
print(f"Número de filas: {df.shape[0]}")
print(f"Número de columnas: {df.shape[1]}")

In [ ]:
# Estadísticas descriptivas
df.describe()

## 3. Manejo de Valores Nulos

Los valores nulos (NaN, None, NA) son uno de los problemas más comunes en datasets reales.

### 3.1 Identificar Valores Nulos

In [ ]:
# Contar valores nulos por columna
print("Valores nulos por columna:")
print(df.isnull().sum())

In [ ]:
# Porcentaje de valores nulos por columna
print("Porcentaje de valores nulos por columna:")
porcentaje_nulos = (df.isnull().sum() / len(df)) * 100
porcentaje_nulos[porcentaje_nulos > 0].sort_values(ascending=False)

In [ ]:
# Visualización de valores nulos
plt.figure(figsize=(12, 6))
sns.heatmap(df.isnull(), cbar=False, yticklabels=False, cmap='viridis')
plt.title('Mapa de Valores Nulos en el Dataset')
plt.xlabel('Columnas')
plt.ylabel('Filas')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### 3.2 Estrategias para Manejar Valores Nulos

Existen varias estrategias:
1. **Eliminar filas/columnas** con valores nulos
2. **Imputar con medidas estadísticas** (media, mediana, moda)
3. **Imputar con valores calculados** (forward fill, backward fill)
4. **Imputar con valores constantes**

In [ ]:
# Crear una copia del dataframe para trabajar
df_clean = df.copy()

print(f"Filas originales: {len(df_clean)}")

#### Estrategia 1: Eliminar filas con valores nulos

In [ ]:
# Ver cuántas filas perderíamos si eliminamos todas las que tienen algún nulo
filas_completas = df_clean.dropna()
print(f"Filas después de eliminar todas con valores nulos: {len(filas_completas)}")
print(f"Filas perdidas: {len(df_clean) - len(filas_completas)}")
print(f"Porcentaje de datos perdidos: {((len(df_clean) - len(filas_completas)) / len(df_clean) * 100):.2f}%")

#### Estrategia 2: Imputación con medidas estadísticas

In [ ]:
# Imputar la columna 'Age' con la mediana (más robusta a outliers que la media)
if 'Age' in df_clean.columns:
    mediana_edad = df_clean['Age'].median()
    print(f"Mediana de edad: {mediana_edad:.2f}")
    df_clean['Age'].fillna(mediana_edad, inplace=True)
    print(f"Valores nulos en 'Age' después de imputación: {df_clean['Age'].isnull().sum()}")

In [ ]:
# Imputar 'Embarked' con la moda (valor más frecuente)
if 'Embarked' in df_clean.columns:
    moda_embarked = df_clean['Embarked'].mode()[0]
    print(f"Moda de Embarked: {moda_embarked}")
    df_clean['Embarked'].fillna(moda_embarked, inplace=True)
    print(f"Valores nulos en 'Embarked' después de imputación: {df_clean['Embarked'].isnull().sum()}")

In [ ]:
# Para columnas con muchos valores nulos (como 'Cabin'), puede ser mejor eliminarlas o crear una categoría
if 'Cabin' in df_clean.columns:
    # Opción 1: Crear una categoría 'Desconocido'
    df_clean['Cabin'].fillna('Desconocido', inplace=True)
    print(f"Valores nulos en 'Cabin' después de imputación: {df_clean['Cabin'].isnull().sum()}")
    
    # O podríamos crear una variable binaria indicando si tiene cabina o no
    df_clean['Tiene_Cabina'] = df_clean['Cabin'].apply(lambda x: 0 if x == 'Desconocido' else 1)
    print("\nDistribución de 'Tiene_Cabina':")
    print(df_clean['Tiene_Cabina'].value_counts())

In [ ]:
# Verificar valores nulos restantes
print("Valores nulos restantes:")
print(df_clean.isnull().sum())

## 4. Detección y Manejo de Duplicados

In [ ]:
# Verificar duplicados completos (todas las columnas iguales)
print(f"Número de filas duplicadas completas: {df_clean.duplicated().sum()}")

In [ ]:
# Ver las filas duplicadas si existen
if df_clean.duplicated().sum() > 0:
    print("Filas duplicadas:")
    print(df_clean[df_clean.duplicated(keep=False)].sort_values(by=df_clean.columns[0]))

In [ ]:
# Verificar duplicados en columnas específicas (por ejemplo, en 'PassengerId')
if 'PassengerId' in df_clean.columns:
    duplicados_id = df_clean.duplicated(subset=['PassengerId']).sum()
    print(f"Duplicados en PassengerId: {duplicados_id}")

In [ ]:
# Eliminar duplicados
filas_antes = len(df_clean)
df_clean = df_clean.drop_duplicates()
filas_despues = len(df_clean)
print(f"Filas eliminadas: {filas_antes - filas_despues}")
print(f"Filas restantes: {filas_despues}")

## 5. Detección y Manejo de Outliers

Los outliers son valores atípicos que se desvían significativamente del resto de los datos.

### Métodos de detección:
1. **Método IQR (Rango Intercuartílico)**
2. **Z-Score**
3. **Visualización (boxplots)**

### 5.1 Visualización de Outliers

In [ ]:
# Seleccionar columnas numéricas
columnas_numericas = df_clean.select_dtypes(include=[np.number]).columns.tolist()
print(f"Columnas numéricas: {columnas_numericas}")

In [ ]:
# Crear boxplots para identificar outliers visualmente
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

for idx, col in enumerate(columnas_numericas[:6]):
    axes[idx].boxplot(df_clean[col].dropna())
    axes[idx].set_title(f'Boxplot de {col}')
    axes[idx].set_ylabel('Valor')

plt.tight_layout()
plt.show()

### 5.2 Método IQR (Rango Intercuartílico)

Un outlier es un valor que está fuera del rango:
- **Límite inferior**: Q1 - 1.5 × IQR
- **Límite superior**: Q3 + 1.5 × IQR

In [ ]:
def detectar_outliers_iqr(df, columna):
    """
    Detecta outliers usando el método IQR.
    
    Parámetros:
    - df: DataFrame
    - columna: nombre de la columna a analizar
    
    Retorna:
    - Serie booleana indicando outliers
    """
    Q1 = df[columna].quantile(0.25)
    Q3 = df[columna].quantile(0.75)
    IQR = Q3 - Q1
    
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    
    print(f"\n--- Análisis de '{columna}' ---")
    print(f"Q1 (25%): {Q1:.2f}")
    print(f"Q3 (75%): {Q3:.2f}")
    print(f"IQR: {IQR:.2f}")
    print(f"Límite inferior: {limite_inferior:.2f}")
    print(f"Límite superior: {limite_superior:.2f}")
    
    outliers = (df[columna] < limite_inferior) | (df[columna] > limite_superior)
    print(f"Número de outliers: {outliers.sum()}")
    
    return outliers

In [ ]:
# Detectar outliers en la columna 'Age'
if 'Age' in df_clean.columns:
    outliers_edad = detectar_outliers_iqr(df_clean, 'Age')
    print(f"\nEdades consideradas outliers:")
    print(df_clean[outliers_edad]['Age'].sort_values())

In [ ]:
# Detectar outliers en la columna 'Fare'
if 'Fare' in df_clean.columns:
    outliers_fare = detectar_outliers_iqr(df_clean, 'Fare')
    print(f"\nTarifas consideradas outliers (primeras 10):")
    print(df_clean[outliers_fare]['Fare'].sort_values(ascending=False).head(10))

### 5.3 Método Z-Score

Un valor se considera outlier si su Z-Score es mayor a 3 o menor a -3.

In [ ]:
def detectar_outliers_zscore(df, columna, umbral=3):
    """
    Detecta outliers usando el método Z-Score.
    
    Parámetros:
    - df: DataFrame
    - columna: nombre de la columna a analizar
    - umbral: valor umbral del Z-Score (por defecto 3)
    
    Retorna:
    - Serie booleana indicando outliers
    """
    media = df[columna].mean()
    std = df[columna].std()
    
    z_scores = np.abs((df[columna] - media) / std)
    
    print(f"\n--- Análisis Z-Score de '{columna}' ---")
    print(f"Media: {media:.2f}")
    print(f"Desviación estándar: {std:.2f}")
    print(f"Umbral Z-Score: {umbral}")
    
    outliers = z_scores > umbral
    print(f"Número de outliers: {outliers.sum()}")
    
    return outliers

In [ ]:
# Detectar outliers en 'Fare' usando Z-Score
if 'Fare' in df_clean.columns:
    outliers_fare_zscore = detectar_outliers_zscore(df_clean, 'Fare')
    print(f"\nTarifas outliers por Z-Score:")
    print(df_clean[outliers_fare_zscore][['Name', 'Fare']].sort_values('Fare', ascending=False))

### 5.4 Estrategias para Manejar Outliers

1. **Eliminar**: Solo si están claramente equivocados
2. **Transformar**: Aplicar logaritmo o raíz cuadrada
3. **Limitar**: Cap/floor a un valor máximo/mínimo
4. **Mantener**: Si son valores válidos y representan casos reales

In [ ]:
# Opción 1: Eliminar outliers extremos
df_sin_outliers = df_clean.copy()

if 'Fare' in df_sin_outliers.columns:
    # Eliminar solo outliers muy extremos usando IQR con factor más alto
    Q1 = df_sin_outliers['Fare'].quantile(0.25)
    Q3 = df_sin_outliers['Fare'].quantile(0.75)
    IQR = Q3 - Q1
    limite_superior = Q3 + 3 * IQR  # Factor 3 en lugar de 1.5 para ser menos agresivos
    
    outliers_extremos = df_sin_outliers['Fare'] > limite_superior
    print(f"Outliers extremos en Fare (>{limite_superior:.2f}): {outliers_extremos.sum()}")
    
    df_sin_outliers = df_sin_outliers[~outliers_extremos]
    print(f"Filas después de eliminar outliers extremos: {len(df_sin_outliers)}")

In [ ]:
# Opción 2: Limitar valores (capping)
df_capped = df_clean.copy()

if 'Fare' in df_capped.columns:
    # Limitar al percentil 95
    limite_percentil_95 = df_capped['Fare'].quantile(0.95)
    print(f"Percentil 95 de Fare: {limite_percentil_95:.2f}")
    
    df_capped['Fare_Capped'] = df_capped['Fare'].clip(upper=limite_percentil_95)
    
    print(f"\nComparación antes y después del capping:")
    print(f"Máximo original: {df_capped['Fare'].max():.2f}")
    print(f"Máximo después de capping: {df_capped['Fare_Capped'].max():.2f}")

In [ ]:
# Visualizar el efecto del capping
if 'Fare' in df_capped.columns:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    axes[0].hist(df_capped['Fare'], bins=50, edgecolor='black')
    axes[0].set_title('Distribución de Fare (Original)')
    axes[0].set_xlabel('Tarifa')
    axes[0].set_ylabel('Frecuencia')
    
    axes[1].hist(df_capped['Fare_Capped'], bins=50, edgecolor='black', color='orange')
    axes[1].set_title('Distribución de Fare (Con Capping)')
    axes[1].set_xlabel('Tarifa')
    axes[1].set_ylabel('Frecuencia')
    
    plt.tight_layout()
    plt.show()

## 6. Resumen y Comparación del Dataset

Comparemos el dataset original con el dataset limpio.

In [ ]:
# Resumen del proceso de limpieza
print("="*60)
print("RESUMEN DEL PROCESO DE LIMPIEZA DE DATOS")
print("="*60)

print(f"\n1. DATASET ORIGINAL:")
print(f"   - Filas: {df.shape[0]}")
print(f"   - Columnas: {df.shape[1]}")
print(f"   - Valores nulos totales: {df.isnull().sum().sum()}")

print(f"\n2. DATASET LIMPIO:")
print(f"   - Filas: {df_clean.shape[0]}")
print(f"   - Columnas: {df_clean.shape[1]}")
print(f"   - Valores nulos totales: {df_clean.isnull().sum().sum()}")

print(f"\n3. CAMBIOS REALIZADOS:")
print(f"   - Filas eliminadas: {df.shape[0] - df_clean.shape[0]}")
print(f"   - Porcentaje de datos retenidos: {(df_clean.shape[0] / df.shape[0] * 100):.2f}%")
print(f"   - Valores nulos eliminados/imputados: {df.isnull().sum().sum() - df_clean.isnull().sum().sum()}")

In [ ]:
# Guardar el dataset limpio
df_clean.to_csv('Titanic-Dataset-Limpio.csv', index=False)
print("\nDataset limpio guardado como 'Titanic-Dataset-Limpio.csv'")

## 7. Conclusiones y Mejores Prácticas

### Lecciones aprendidas:

1. **Valores Nulos:**
   - Siempre analizar el porcentaje de valores nulos antes de decidir qué hacer
   - Usar la mediana para variables numéricas (más robusta a outliers)
   - Usar la moda para variables categóricas
   - Considerar crear variables indicadoras para valores faltantes

2. **Duplicados:**
   - Verificar duplicados tanto completos como por columnas clave
   - Entender por qué existen duplicados antes de eliminarlos

3. **Outliers:**
   - No siempre los outliers deben eliminarse - pueden ser valores válidos
   - Usar visualizaciones para entender la naturaleza de los outliers
   - Considerar transformaciones (log, sqrt) antes de eliminar
   - El método IQR es más robusto que Z-Score para distribuciones no normales

4. **General:**
   - Documentar todas las decisiones de limpieza
   - Mantener una copia del dataset original
   - Validar que la limpieza no introdujo sesgos
   - Considerar el contexto del negocio/dominio

## 8. Ejercicios Prácticos

### Ejercicio 1:
Crea una función que automatice el proceso de limpieza para cualquier dataset, incluyendo:
- Detección de valores nulos
- Imputación automática según el tipo de dato
- Detección de outliers

### Ejercicio 2:
Analiza cómo afecta la eliminación de outliers a las estadísticas descriptivas del dataset.

### Ejercicio 3:
Compara diferentes métodos de imputación (media vs mediana vs moda) y evalúa cuál es más apropiado para cada columna.